In [1]:
alphabet = dict() #for each letter save amount of pages and link to the first of these pages
alphabet['a'] = [37, "https://cognates.ru/words"]
alphabet['b'] = [42, "https://cognates.ru/words/b"]
alphabet['v'] = [66, "https://cognates.ru/words/v"]
alphabet['g'] = [23, "https://cognates.ru/words/g"]
alphabet['d'] = [35, "https://cognates.ru/words/d"]
alphabet['e'] = [3, "https://cognates.ru/words/e"]
alphabet['yo'] = [1, "https://cognates.ru/words/yo"]
alphabet['zh'] = [7, "https://cognates.ru/words/zh"]
alphabet['z'] = [37, "https://cognates.ru/words/z"]
alphabet['i'] = [21, "https://cognates.ru/words/i"]
alphabet['yi'] = [1, "https://cognates.ru/words/yi"]
alphabet['k'] = [57, "https://cognates.ru/words/k"]
alphabet['l'] = [18, "https://cognates.ru/words/l"]
alphabet['m'] = [36, "https://cognates.ru/words/m"]
alphabet['n'] = [47, "https://cognates.ru/words/n"]
alphabet['o'] = [60, "https://cognates.ru/words/o"]
alphabet['p'] = [142, "https://cognates.ru/words/p"]
alphabet['r'] = [42, "https://cognates.ru/words/r"]
alphabet['s'] = [78, "https://cognates.ru/words/s"]
alphabet['t'] = [30, "https://cognates.ru/words/t"]
alphabet['u'] = [22, "https://cognates.ru/words/u"]
alphabet['f'] = [15, "https://cognates.ru/words/f"]
alphabet['h'] = [10, "https://cognates.ru/words/h"]
alphabet['cz'] = [5, "https://cognates.ru/words/cz"]
alphabet['ch'] = [10, "https://cognates.ru/words/ch"]
alphabet['sh'] = [12, "https://cognates.ru/words/sh"]
alphabet['sc'] = [2, "https://cognates.ru/words/sc"]
alphabet['iy'] = [1, "https://cognates.ru/words/iy"]
alphabet['ye'] = [12, "https://cognates.ru/words/ye"]
alphabet['yu'] = [2, "https://cognates.ru/words/yu"]
alphabet['ya'] = [4, "https://cognates.ru/words/ya"]

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

def get_children(link_str):
    page = urlopen(link_str)
    soup = BeautifulSoup(page)

    links = soup.find_all(attrs={"class":"list"})
    if len(links):
        arr = str(links[0])
        arr = arr.replace("</a>", "")
        arr = arr.replace("</p>", "")
        arr = arr.replace('<p class="list">', "")
        arr = arr.replace("<a href=", "")
        arr = arr.replace(">", " ")
        curr_list = arr.split()

        children = []
        for i in range(len(curr_list) // 2):
            children.append([curr_list[2 * i].replace('"', ''), curr_list[2 * i + 1]])
        return children
    return []

In [5]:
import csv

MAXN = 10

file = open('cognates_full.csv', 'w')
writer = csv.writer(file)
writer.writerows([['first_word', 'second_word']])

with file:
    first_word = list()
    second_word = list()
    for key in alphabet:
        for i in range(alphabet[key][0]):
            curr_link = alphabet[key][1]
            if i != 0:
                curr_link += "/pn"
                curr_link += str(i + 1)
            word_list = get_children(curr_link)
            for elem in word_list:
                child_page = "https://cognates.ru" + elem[0]
                cognates = get_children(child_page)
                for i in range(min(MAXN, len(cognates))):
                    cognate = cognates[i]
                    first_word.append(elem[1])
                    second_word.append(cognate[1])
                    writer = csv.writer(file)
                    writer.writerows([[elem[1], cognate[1]]])
print("Writing complete")

Writing complete


In [6]:
import pandas as pd

data = pd.read_csv('cognates_full.csv', encoding='utf_8')
data.head()

,first_word,second_word
0,А,А-конто
1,А,Аканье
2,А,Акать
3,А,Акающий
4,А,Ашка


In [7]:
def dist_Levenshtein(s1, s2):
    d = [[0] * (len(s2) + 1) for _ in range(len(s1) + 1)]
    for i in range(len(s1) + 1):
        for j in range(len(s2) + 1):
            if i == 0:
                d[i][j] = j
            elif j == 0:
                d[i][j] = i
            else:
                d[i][j] = min(d[i][j - 1] + 1, d[i - 1][j] + 1)
                delta = 0 if s1[i - 1] == s2[j - 1] else 1
                d[i][j] = min(d[i][j], d[i - 1][j - 1] + delta)
                
    return d[len(s1)][len(s2)]

In [8]:
print(dist_Levenshtein('Levenshtein', 'Lenvinsten'))

4


In [9]:
def longest_common_substring(s1, s2):
    d = [[0] * (len(s2) + 1) for _ in range(len(s1) + 1)]
    max_val = 0
    for i in range(len(s1) + 1):
        for j in range(len(s2) + 1):
            if i and j and s1[i - 1] == s2[j - 1]:
                d[i][j] = d[i - 1][j - 1] + 1
            if d[i][j] > max_val:
                max_val = d[i][j]

    return max_val

In [10]:
print(longest_common_substring('subsequence', 'subeuencs'))

4


In [11]:
data['Levenshtein'] = data.apply(lambda row: dist_Levenshtein(row.first_word, row.second_word), axis=1)
data.head()

,first_word,second_word,Levenshtein
0,А,А-конто,6
1,А,Аканье,5
2,А,Акать,4
3,А,Акающий,6
4,А,Ашка,3


In [12]:
data['longest_common_substr'] = data.apply(lambda row: longest_common_substring(row.first_word, row.second_word), axis=1)
data.head()

,first_word,second_word,Levenshtein,longest_common_substr
0,А,А-конто,6,1
1,А,Аканье,5,1
2,А,Акать,4,1
3,А,Акающий,6,1
4,А,Ашка,3,1


In [13]:
data.to_csv('cognates_dataset.csv', encoding='cp1251')

# Очистка данных и новая метрика

In [1]:
def longest_common_subsequence(s1, s2):
    d = [[0] * (len(s2) + 1) for _ in range(len(s1) + 1)]
    max_val = 0
    for i in range(len(s1) + 1):
        for j in range(len(s2) + 1):
            if i and j and s1[i - 1] == s2[j - 1]:
                d[i][j] = d[i - 1][j - 1] + 1
            if i and j and s1[i - 1] != s2[j - 1]:
                d[i][j] = max(d[i - 1][j], d[i][j - 1])
            if d[i][j] > max_val:
                max_val = d[i][j]

    return max_val

In [2]:
import pandas as pd

data = pd.read_csv('cognates_dataset.csv', encoding='cp1251')
data.head()

,Unnamed: 0,first_word,second_word,Levenshtein,longest_common_substr
0,0,А,А-конто,6,1
1,1,А,Аканье,5,1
2,2,А,Акать,4,1
3,3,А,Акающий,6,1
4,4,А,Ашка,3,1


In [3]:
data.pop('Unnamed: 0')
data.head()

,first_word,second_word,Levenshtein,longest_common_substr
0,А,А-конто,6,1
1,А,Аканье,5,1
2,А,Акать,4,1
3,А,Акающий,6,1
4,А,Ашка,3,1


In [4]:
data['longest_common_subs'] = data.apply(lambda row: longest_common_subsequence(row.first_word, row.second_word), axis=1)
data.head()

,first_word,second_word,Levenshtein,longest_common_substr,longest_common_subs
0,А,А-конто,6,1,1
1,А,Аканье,5,1,1
2,А,Акать,4,1,1
3,А,Акающий,6,1,1
4,А,Ашка,3,1,1


In [5]:
import spacy
from nltk.corpus import stopwords

nlp = spacy.load('ru_core_news_lg')
stops = stopwords.words("russian")

In [6]:
stop_pos_tags = ["ADP", "CCONJ", "PRON", "DET", "SCONJ", "AUX"]

bad_indexes = []

for index, row in data.iterrows():
    condition1 = row.first_word in stops or nlp(row.first_word)[0].pos_ in stop_pos_tags
    condition2 = row.second_word in stops or nlp(row.second_word)[0].pos_ in stop_pos_tags
    if condition1 or condition2:
        bad_indexes.append(index)


In [10]:
print(len(bad_indexes))

1191


In [16]:
clean_data = data.drop(data.index[bad_indexes])
clean_data = clean_data.reset_index(drop=True)
clean_data.head()

,first_word,second_word,Levenshtein,longest_common_substr,longest_common_subs
0,А-конто,Аканье,4,1,3
1,А-конто,Акать,4,1,3
2,А-конто,Акающий,6,1,2
3,А-конто,Ашка,5,1,2
4,А-конто,Бамовец,6,1,1


In [17]:
clean_data.to_csv('cognates_dataset_clean.csv', encoding='cp1251')